In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import geocoder

** Obtain the Data **

In [8]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find("table")
df = pd.read_html(str(table))
df1 = pd.DataFrame(df[0])

** Ignore cells with a borough that is Not assigned. **

In [9]:
df1 = df1[df1.Borough != "Not assigned"]

** More than one neighborhood can exist in one postal code area. **

In [10]:
df1 = df1.groupby(['Postcode', 'Borough'], as_index=False).agg({
    'Neighbourhood': lambda x: ', '.join(x)
})

** If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  **

In [11]:
df1.Neighbourhood[df1.Neighbourhood == "Not assigned"] = df1.Borough

In [12]:
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


** Use the .shape method to print the number of rows of your dataframe.**

In [12]:
df1.shape

(103, 3)

** Get the latitude and the longitude coordinates of each neighborhood **

In [13]:
geo = pd.read_csv("Geospatial_Coordinates.csv")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_merge = pd.merge(df1, geo, left_on = "Postcode", right_on = "Postal Code")
df_merge.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
